## Import numpy and matplotlib:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Linear Regression with Gradient Descent for a given loss function

Next, let us create a class of Linear Regression that performs gradient descent for a given loss function. 

Exercise: **Please check the implementation and if it fits what we discussed about in class** --> Ask questions if this is not the case! 

In [ ]:
class LinearRegressionGradientDescent():
    """
    Linear regression implementation (gradient descent)
    """

    def __init__(self, loss, eta=0.1, n_steps=5):
        """Initializes the linear regression model.

        Args:
            loss: Loss function object that computes the loss and its gradient.
            eta (float, optional): Learning rate. Defaults to 0.1.
            n_steps (int, optional): Number of steps. Defaults to 5.
        """
        
        self.eta = eta
        self.n_steps = n_steps
        self.loss = loss
            
    def fit(self, X, y):
        """
        Fits the linear regression model.

        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        """        

        # make sure that we have multidimensional np arrays
        X = np.array(X).reshape((X.shape[0], -1))
        # IMPORTANT: Make sure that we have a column vector! 
        y = np.array(y).reshape((len(y), 1))
        
        # starting point
        w = np.zeros((X.shape[1], 1))
        
        # gradient descent steps
        for i in range(self.n_steps):

            grad = self.loss.gradient(X, y, w)
            w = w - self.eta * grad
        self._w = w
                
    def predict(self, X):
        """
        Computes predictions for a new set of points.

        Parameters
        ----------
        X : Array of shape [n_samples, n_features]

        Returns
        -------
        predictions : Array of shape [n_samples, 1]
        """                     

        # make sure that we have multidimensional np arrays
        X = np.array(X).reshape((X.shape[0], -1))        

        # compute predictions
        predictions = np.dot(X, self._w)

        return predictions

## Loss functions

Next, we define the loss functions from the slides. 

Exercise: **Please verify that the forward-methods of all classes really calculate the respective loss functions**

You do not have to directly understand, why the gradient method works as given here. But you should know, why we need a gradient method in all loss classes! 

In [ ]:
## MSE loss

class MSELoss():
    def __init__(self):
        pass

    def forward(self, target, predictions):
        """
        Computes the mean squared error loss.

        Parameters
        ----------
        target : Array of shape [n_samples, 1]
        predictions : Array of shape [n_samples, 1]

        Returns
        -------
        loss : float
        """                     

        loss = np.mean((target - predictions) ** 2)

        return loss
    
    def gradient(self, X, y, w):
        """
        Computes the gradient of the mean squared error loss.

        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        w : Array of shape [n_features, 1]

        Returns
        -------
        gradient : Array of shape [n_features, 1]
        """                     

        n = X.shape[0]
        gradient = 2/n * np.dot(X.T, (np.dot(X, w) - y))

        return gradient



In [ ]:
## MAE loss

class MAELoss():
    def __init__(self):
        pass

    def forward(self, target, predictions):
        """
        Computes the mean absolute error loss.

        Parameters
        ----------
        target : Array of shape [n_samples, 1]
        predictions : Array of shape [n_samples, 1]

        Returns
        -------
        loss : float
        """                     

        loss = np.mean(np.abs(target - predictions))

        return loss
    
    def gradient(self, X, y, w):
        """
        Computes the gradient of the mean absolute error loss.

        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        w : Array of shape [n_features, 1]

        Returns
        -------
        gradient : Array of shape [n_features, 1]
        """                     

        n = X.shape[0]
        gradient = 1/n * np.dot(X.T, np.sign(np.dot(X, w) - y))

        return gradient

In [ ]:
## Huber loss

class HuberLoss():
    def __init__(self, delta=1):
        self.delta = delta

    def forward(self, target, predictions):
        """
        Computes the Huber loss.

        Parameters
        ----------
        target : Array of shape [n_samples, 1]
        predictions : Array of shape [n_samples, 1]

        Returns
        -------
        loss : float
        """                     

        loss = np.mean(np.where(np.abs(target - predictions) < self.delta, 0.5 * (target - predictions) ** 2, self.delta * np.abs(target - predictions) - 0.5 * self.delta ** 2))

        return loss
    
    def gradient(self, X, y, w):
        """
        Computes the gradient of the Huber loss.

        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        w : Array of shape [n_features, 1]

        Returns
        -------
        gradient : Array of shape [n_features, 1]
        """                     

        n = X.shape[0]
        gradient = np.dot(X.T, np.where(np.abs(np.dot(X, w) - y) < self.delta, np.dot(X, w) - y, self.delta * np.sign(np.dot(X, w) - y)))

        return gradient

In [ ]:
## epsilon-insensitive loss

class EpsilonInsensitiveLoss():
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon

    def forward(self, target, predictions):
        """
        Computes the epsilon-insensitive loss.

        Parameters
        ----------
        target : Array of shape [n_samples, 1]
        predictions : Array of shape [n_samples, 1]

        Returns
        -------
        loss : float
        """                     

        loss = np.mean(np.maximum(0, np.abs(target - predictions) - self.epsilon))

        return loss
    
    def gradient(self, X, y, w):
        """
        Computes the gradient of the epsilon-insensitive loss.

        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        w : Array of shape [n_features, 1]

        Returns
        -------
        gradient : Array of shape [n_features, 1]
        """                     

        n = X.shape[0]
        gradient = np.dot(X.T, np.where(np.abs(np.dot(X, w) - y) > self.epsilon, np.sign(np.dot(X, w) - y), 0))

        return gradient

In [ ]:
## Pinball loss

class PinballLoss():
    def __init__(self, tau=0.5):
        self.tau = tau

    def forward(self, target, predictions):
        """
        Computes the pinball loss.

        Parameters
        ----------
        target : Array of shape [n_samples, 1]
        predictions : Array of shape [n_samples, 1]

        Returns
        -------
        loss : float
        """                     

        loss = np.mean(np.where(target <= predictions, (1-self.tau) * (predictions - target), self.tau * (target - predictions)))

        return loss
    
    def gradient(self, X, y, w):
        """
        Computes the gradient of the pinball loss.

        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        w : Array of shape [n_features, 1]

        Returns
        -------
        gradient : Array of shape [n_features, 1]
        """                     

        n = X.shape[0]
        gradient = np.dot(X.T, np.where(np.dot(X, w) >= y, 1-self.tau, - self.tau))

        return gradient

## Train a constant with different losses

In [ ]:
np.random.seed(42)  # Set seed for reproducibility
y = np.random.exponential(scale=2.0, size=1000)

## For testing purposes at home, you can also use other datasets:
#y = np.random.lognormal(size= 1000, mean=0, sigma=0.5)
# y = np.random.chisquare(df=2, size=1000)

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(y, bins=50, color='blue', alpha=0.5)
plt.grid()
plt.title('Histogram of y')
plt.xticks(np.arange(0, 20, 0.5), rotation = 90)
plt.show()

Exercise: **Try to fit a constant without computer**:
What would you guess, which constant will be fitted using 
- mean squared error
- mean absolute error
- Huber-loss with $\delta = 1$
- $\epsilon$-insensitive loss with $\epsilon = 0.1$
- $\epsilon$-insensitive loss with $\epsilon = 1$
- pinball-loss with $\tau = 0.25$
- pinball-loss with $\tau = 0.75$
Write down your answers in the following cell:

In [ ]:
w_guess_mse =  ## Guess for MSE loss
w_guess_mae =  ## Guess for MAE loss
w_guess_huber =  ## Guess for Huber loss with delta=1
w_guess_epsilon01 =  ## Guess for epsilon-insensitive loss with epsilon=0.1
w_guess_epsilon1 =  ## Guess for epsilon-insensitive loss with epsilon=1
w_guess_pinball025 =  ## Guess for pinball loss with tau=0.25
w_guess_pinball075 =  ## Guess for pinball loss with tau=0.75

Exercise: **Complete the method fit_constant(loss_fn, y) below**

In [ ]:
def fit_constant(loss_fn, y):
    """
    Fits a constant model using the given loss function.

    Parameters
    ----------
    loss_fn : Loss function object
    y : Array of shape [n_samples, 1]
    """

    # create a constant model
    X = 

    # fit the model using the loss function
    eta = 0.001
    n_steps = 10000
    model = 
    model.fit(...)

    return model._w[0][0]

Now we can perform gradient descent in order to fit all constants:

In [ ]:
loss_names = ['MSE', 'MAE', 'Huber', 'EpsilonInsensitive (epsilon=0.1)', 'EpsilonInsensitive (epsilon=1)', 'Pinball (tau=0.25)', 'Pinball (tau=0.75)']
loss_functions = [MSELoss(), MAELoss(), HuberLoss(delta=1), EpsilonInsensitiveLoss(epsilon=0.1), EpsilonInsensitiveLoss(epsilon=1), PinballLoss(tau=0.25), PinballLoss(tau=0.75)]
loss_guesses = [w_guess_mse, w_guess_mae, w_guess_huber, w_guess_epsilon01, w_guess_epsilon1, w_guess_pinball025, w_guess_pinball075] 
constant_pred = []
for f in range(len(loss_functions)):
    loss_fn = loss_functions[f]
    loss_name = loss_names[f]
    loss_guess = loss_guesses[f]
    w = fit_constant(loss_fn, y)
    constant_pred.append(w)
    print(f"Constant model using {loss_name}: {w:.2f}")
    print(f"Guess for {loss_name}: {loss_guess:.2f}")

Let us also plot the fitted constants. Is it in line with your expectation?

In [ ]:
colors = ['blue', 'orange', 'green', 'red', 'black', 'pink', 'purple']
plt.figure(figsize=(10, 5))
plt.hist(y, bins=50, color='blue', alpha=0.5)
for f in range(len(loss_functions)):
    loss_name = loss_names[f]
    plt.axvline(x=constant_pred[f], color = colors[f], linestyle='--', label=f'{loss_name} prediction: {constant_pred[f]:.2f}')
plt.title('Constant model predictions using different loss functions')
plt.xlabel('y')
plt.ylabel('Frequency')
plt.legend()
plt.show()

## Create Dataset with outlier

Now we create a dataset with an outlier

In [ ]:
# generate some artificial data to be fitted
m = 20
w0_true = 4
w1_true = 2
w = np.array([w0_true, w1_true])

X = np.linspace(-1,1,m).reshape(m, 1)
ones = np.ones((X.shape[0], 1))
X = np.concatenate((ones, X), axis=1)

y = np.dot(X, w) + np.random.randn(m) * 0.4

y[0] += 5

In [ ]:
plt.figure()
plt.scatter(X[:,-1], y, color='blue')
plt.title('Data with outlier')
plt.show()

## Try out different loss functions for our dataset 

Exercise: **Try out different loss functions for this dataset and plot the values and the fitted line. What do you observe?** 

In [ ]:
for f in range(len(loss_functions)):
    loss_fn = loss_functions[f]
    loss_name = loss_names[f]
    model = LinearRegressionGradientDescent(loss_fn, eta=0.001, n_steps=10000)
    model.fit(X, y)

    plt.figure()
    plt.scatter(X[:,-1], y, color='blue')
    plt.plot(X[:,-1], model.predict(X), color='red')
    plt.title(f'{loss_name} prediction')
    plt.xlabel('X')
    plt.ylabel('y')
plt.show()

Exercise: **Remove the outlier from the dataset. Can you still see differences between MSE, MAE, Huber-loss, $\epsilon$-insensitive loss and Pinball loss? 

Exercise: **Now create a dataset with two outliers (one high and one low outlier). What does happen?**